<a href="https://colab.research.google.com/github/KaiaX926/P-ADS-Transpotation-NYC/blob/main/Citi_Bike_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import requests
from lxml import etree
from io import BytesIO
from zipfile import ZipFile
import urllib.request
import pandas as pd

class ScrapeData():
    def __init__(self):
        self.link = requests.get('https://s3.amazonaws.com/tripdata')
        self.source = etree.fromstring(self.link.content)
        self.lists = []
        self.table = 0
    
    def Scrape(self):
        for item in self.source.xpath('//*'):
            if item.text and item.text.endswith('zip') \
            and (item.text[:6] >= '202109' and item.text[:6] <= '202202' and item.text[:2] == '20'):
                origin = 'https://s3.amazonaws.com/tripdata/' + item.text
                print(item.text)
                self.table += 1
                url = urllib.request.urlopen(origin)
                with ZipFile(BytesIO(url.read())) as my_zip_file:
                    for contained_file in my_zip_file.namelist():
                        for line in my_zip_file.open(contained_file).readlines():
                            try:
                                l = line[:-2].decode("utf-8")
                                l = l.replace('"', "").split(",")
                                if self.table > 1 and l[0] == 'ride_id':
                                    pass
                                else:
                                    self.lists.append(l)
                            except:
                                pass
        df = pd.DataFrame(self.lists)
        df.columns = df.iloc[0]
        df = df[1:]
        
        df.to_csv('220902.csv', index=False) 
        
        return df

In [3]:
S = ScrapeData()
df_220902 = S.Scrape()

202109-citibike-tripdata.csv.zip
202110-citibike-tripdata.csv.zip
202111-citibike-tripdata.csv.zip
202112-citibike-tripdata.csv.zip
202201-citibike-tripdata.csv.zip
